In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [4]:
base_url='https://www.cian.ru/cat.php'
req_param='deal_type=sale&offer_type=flat&region=184723'

---

In [28]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [29]:
# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-08 12:56:59 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-08 12:56:59 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-08 12:57:02 | AdsListParser: downloader init
[INFO    ] 2022-09-08 12:57:02 | CianParser: downloader init
[INFO    ] 2022-09-08 12:57:02 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-08 12:57:09 | AdsListParser: read page 53
[INFO    ] 2022-09-08 12:57:19 | AdsListParser: read page 54
[INFO    ] 2022-09-08 12:57:19 | AdsListParser: last page detected
[INFO    ] 2022-09-08 12:57:19 | DownloaderSeleniumFirefox: close virtual browser
53


In [30]:
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)

In [31]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'tmp/cian_{ts}_raw.xlsx',index=False)

---

In [ ]:
# 'avito_id','title','price','obj_name','adr','description','page','ts'

In [61]:
# df.sample()
# df.columns

In [36]:
# df[['OfferTitle','OfferSubtitle']]
# df['GeoLabel'].apply(lambda s: ', '.join(s) )

In [48]:
# df['MainPrice'].apply( lambda s: int( re.sub(r'\D','',s) ) )

----

In [60]:
from lib.cian import CianDataCleaner

CianDataCleaner.transform(df)

,title,obj_name,adr,nrooms,floor,nfloors,area,is_studio,is_apartment,is_part,...,is_openspace,is_roof,is_SNT,price,priceM,is_last_floor,cian_url,description,page,ts
0,"1-комн. кв., 30,8 м², 3/5 этаж",,"Севастополь, р-н Нахимовский, мкр. Горпищенко, улица Горпищенко, 94",1,3,5,30.80,False,False,False,...,False,False,False,4700000,4.70,False,https://sevastopol.cian.ru/sale/flat/266873983/,"Однокомнатная квартира 30,8 м2 на 3/5 этаже. Расположена в г. Севастополь, Нахимовский район, ул. Горпищенко, д.94. Эта квартира идеально подойдёт как для личного проживания, так и для сдачи в аренду.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Нахимовский район, ул. Горпищенко, д.94.\n-Год постройки дома: 1986 г.\n-Этаж/этажность: 3/5\n-Количество комнат: 1\n-Комнаты: изолированные\n-Общая площадь: 30,8 м2\n-Жилая площадь: 13,3 м2\n-Площадь кухни: 7,1 м2\n-Высота потолка: 2,6 м\n-Санузел: совмещенный\n-Водоснабжение: центральное\n-Горячее водоснабжение: бойлер\n-Отопление: центральное\nДополнительная информация: \nДом галерейного типа. Установлены пластиковые окна. Центральное отопление, бойлер, электрическая плита с пониженным коэффициентом за коммунальные платежи, застекленная лоджия.\n \nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nИз окон открывается вид на город и море. До центра города 20 минут на автобусе.\nПодходит для наличного расчета и под гражданскую ипотеку, материнский капитал, жилищные сертификаты.\nДокументы РФ, проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять! \n \nВозможно произошел сбой и вы не смогли дозвониться. Напишите ваш номер мы свяжемся с вами в ближайшее время или попробуйте позвонить еще раз",53,2022-09-08 12:57:19.031678
1,"2-комн. кв., 40 м², 2/2 этаж",,"Севастополь, р-н Гагаринский, мкр. Фиолент, Орбита СТ , 168",2,2,2,40.00,False,False,False,...,False,False,True,4700000,4.70,True,https://sevastopol.cian.ru/sale/flat/272215344/,"ЗАЕЗЖАЙ И ЖИВИ!!!!\nПродам 2-х комнатную квартиру в Гагаринском районе г. Севастополь. В квартире сделан полностью евроремонт. Установлена медная проводка, светильники, качественный ламинат, новая мебель, душевая кабина, раковина с тумбой, стиральная машина, холодильник. \nКухня просторная и светлая общей площадью 12м.кв. \nКаждая комната по 12 и 13-м.кв.\nПри продаже вся мебель и техника остаётся!!!!\nВо дворе дома будет установлена мангальная зона, а территория устелена тротуарной плиткой.\nКвартира находится в двухэтажном доме на Фиоленте, в двух минутах от остановки общественного транспорта. Рядом знаменитый красивый пляж ""Баути"" (возле маяка). \nК гипермаркету МЕТРО на машине - 15минут езды, там же школы и детские сады! \nЭта квартира подойдет как для постоянного проживания, так и под сдачу в аренду!\nДокументы все Российского образца.\nНомер объекта: 95424",53,2022-09-08 12:57:19.031678
2,"Студия, 33,5 м², 4/16 этаж",,"Севастополь, р-н Гагаринский, мкр. 5-й, улица Александра Маринеско, 1б/2",0,4,16,33.50,True,False,False,...,False,False,False,4700000,4.70,False,https://sevastopol.cian.ru/sale/flat/272634427/,"Квартира студия 31.51 м², 4/16 эт., в новом, сданном ЖК Гагаринские Высотки.\nЖилой комплекс расположен в Гагаринском районе города Севастополя, ул.Александра Маринеско 1Б/2.\n\nКвартира светлая, расположение окон на южную сторону. Есть вид на море и закаты.\n\nСтудия 31.51 м²:\n* Коридор - 4.43 м²,\n* Ниша под гардероб - 1.06 м²,\n* Совмещенный санузел - 4.42 м²,\n* Комната, объединённая с кухонной нишей - 20.75 м²,\n* Лоджия с панорамным остеклением - 2.82 м².\nВысота потолков 2.75 м.\n\nВ квартире установлено: металлическая входная дверь с замком, металлопластиковые окна с подоконниками и балконные двери, прибор учета воды, радиаторы отопления, счетчик учета электроэнергии в поэтажном щите учета, теплосчетчик на лестничной клетке перед входом в квартиру.\n\nВ квартире выполнены следующие работы

----

In [ ]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])

---

In [ ]:
# from lib.downloader import DownloaderSimple
# html = DownloaderSimple().get(url)
# with open('tmp/cian_.html','wt') as f: f.write(html[0])

---

In [ ]:
# <article data-name="CardComponent" 

In [ ]:
# <span data-mark="OfferTitle"
# <span data-mark="OfferSubtitle" 
# <span data-mark="Deadline"

# <a data-name="GeoLabel"

# <span data-mark="MainPrice"

# <p data-mark="PriceInfo" 

# <div data-name="Description" 

# <div data-name="LinkArea" class="_93444fe79c--container--kZeLu _93444fe79c--link--DqDOy">
# <a href="https://sevastopol.cian.ru/sale/flat/273181085/" class="_93444fe79c--link--eoxce">

# <div data-name="TimeLabel" 

In [ ]:
# def get_link(tag):
#     return tag.find('a').attrs['href']

# def get_ts(tag):
#     return [ t.text for t in tag.find_all('span') ]

# def get_adr(tag):
#     return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]

# def parse_item(tag):
#     return {
#     'OfferTitle':tag.find('span',{'data-mark':'OfferTitle',}).text,
    
#     #'OfferSubtitle': tag.find('span',{'data-mark':'OfferSubtitle',}).text,
#     # 'Deadline': tag.find('span',{'data-mark':'Deadline',}).text,
    
#     'GeoLabel':  get_adr(tag),  # tag.find('span',{'data-name':'GeoLabel',}).text,
        
#     'MainPrice': tag.find('span',{'data-mark':'MainPrice',}).text,
#     'PriceInfo': tag.find('p',{'data-mark':'PriceInfo',}).text,
#     'Description': tag.find('div',{'data-name':'Description',}).text,
#     'LinkArea': get_link( tag.find('div',{'data-name':'LinkArea',})), 
#     'TimeLabel': get_ts( tag.find('div',{'data-name':'TimeLabel',})),  
#     }
    
    

In [ ]:
# root = BeautifulSoup(html,'html.parser')
# text = [
#     parse_item(tag)
#     for tag in root.find_all('article',{'data-name':'CardComponent',}) 
# ]


In [ ]:
# pd.DataFrame(text)

In [ ]:
# text[0]

In [ ]:
# text[1]

In [ ]:
# from lib.downloader import DownloaderSimple

# df,html = CianParser(driver=DownloaderSimple()).load(req_param, keep_html=True, page_limit=3)

# # html = DownloaderSimple().get(url)
# with open('tmp/cian.html','wt') as f: f.write(html)

In [ ]:
# class CianDataCleaner:
    
#     @staticmethod
#     def transform(data):
#         df = data.copy()
#         df = df.rename(
#             columns={
#                 'OfferTitle':'title',
#                 'OfferSubtitle':'obj_name',
#                 'Description':'description',
#                 'LinkArea':'cian_url',
#             }
#         )
        
#         df['nrooms'] = df['title'].str.extract( r'.*(\d)-к.*',expand=False)
#         df['floor'] = df['title'].str.extract( r'.*(\d+)/\d+.эт.*',expand=False)
#         df['nfloors'] = df['title'].str.extract( r'.*\d+/(\d+).эт.*',expand=False)
#         df['area'] = (
#             df['title']
#             .str.extract( r'.*, ([\d,]+).*м²,.*',expand=False)
#             .str.replace(',','.')
#         )

#         df['is_studio'] = df['title'].str.lower().str.match(r'.*студи.*')
#         df['is_apartment'] = df['title'].str.lower().str.match(r'.*апартамент.*')
#         df['is_part'] = df['title'].str.lower().str.match(r'.*дол.*')
#         df['is_auction'] = df['title'].str.lower().str.match(r'.*аукци.*')
#         df['is_openspace'] = df['title'].str.lower().str.match(r'.*своб.*планир.*')
        
#         df['is_roof'] = df['description'].str.lower().str.match('.*мансард.*')
        
#         df['adr'] = df['GeoLabel'].apply(lambda s: ', '.join(s) )
        
#         df['is_SNT'] = (
#             df['adr'].str.match('.*СТ .*') 
#             | df['adr'].str.match('.*СНТ .*') 
#             | df['adr'].str.lower().str.match('.*садов.*')
#         )

#         df['adr'] = (
#             df['adr'] 
#                 .apply(lambda s: re.sub(r'(\d)(к\d)',r'\1 \2',s))
#                 .apply(lambda s: re.sub(r'\bш\.',' шоссе ',s) )
#                 .apply(lambda s: re.sub(r'\bпр-т\b',' проспект ',s) )
#                 .apply(lambda s: re.sub(r'\bпер\.',' перeулок ',s) )
#                 .apply(lambda s: re.sub(r'\bб-р\b',' бульвар ',s) )
#                 .apply(lambda s: re.sub(r'\bс\.',' село ',s) )
#                 .apply(lambda s: re.sub(r'\bпос\. городского типа\b.',' ',s) )
#                 .apply(lambda s: re.sub(r'\bпос[её]лок городского типа\b.',' ',s) )
#                 .apply(lambda s: re.sub(r'\bнаб\.',' набережная ',s) )
#                 .apply(lambda s: re.sub(r'\bпл\.',' площадь ',s) )
#                 .apply(lambda s: re.sub(r'\bул\.',' улица ',s) )
#                 .apply(lambda s: re.sub(r'\bпос\.',' посёлок ',s) )
#                 .apply(lambda s: re.sub(r'\bпр\.',' проезд ',s) )
#                 .apply(lambda s: re.sub(r'\bпр-д',' проезд ',s) )
#                 .apply(lambda s: re.sub(r'садоводческое некоммерческое товарищество',' СНТ ',s) )
#                 .apply(lambda s: re.sub(r'садовое некоммерческое товарищество',' СНТ ',s) )
#                 .apply(lambda s: re.sub(r'садоводческое товарищество',' СТ ',s) )
#                 .apply(lambda s: re.sub(r'садовое товарищество',' СТ ',s) )
#                 .apply(lambda s: re.sub(r',\s*,',', ',s))
#                 .apply(lambda s: re.sub(r'  +',' ',s))
#             )

#         # df['avito_id'] = df['avito_id'].astype(int)

#         df['price'] = df['MainPrice'].apply( lambda s: int( re.sub(r'\D','',s) ) )
#         df['nrooms'] = df['nrooms'].fillna('0').astype(int)
#         df['floor'] = df['floor'].fillna('0').astype(int)
#         df['nfloors'] = df['nfloors'].fillna('0').astype(int)
#         df['area'] = df['area'].fillna('0.').astype(float)
#         df['priceM'] = df['price']/1e6
#         df['is_last_floor'] = ( df['floor'] == df['nfloors'] )

#         cols=[
#             'title',
#             'obj_name',
#             'adr',
#             'nrooms',
#             'floor',
#             'nfloors',
#             'area',
#             'is_studio',
#             'is_apartment',
#             'is_part',
#             'is_auction',
#             'is_openspace',
#             'is_roof',
#             'is_SNT',
#             'price',
#             'priceM',
#             'is_last_floor',
#             'cian_url',
#             'description',
#             'page',
#             'ts',

#             # 'OfferSubtitle',
#             # 'Deadline',
#             # 'MainPrice',
#             # 'PriceInfo',
#             # 'GeoLabel',
#             # 'TimeLabel',

#             ]
#         return df[cols]


In [26]:
# class CianParser(AdsListParser):
    
#     def __init__(self,driver,base_url='https://www.cian.ru/cat.php',random_delay=10.,):
#         super().__init__(
#             driver=driver, 
#             base_url=base_url,
#             item_tag=['article',{'data-name':'CardComponent',},],
#             paginator_url_param='p',
#         )
#         self._random_delay = random_delay
#         logging.info('CianParser: downloader init')
    
#     def _read_page(self,url,npage=1): 
#         logging.debug(f'CianParser: {url}')
#         if self._random_delay>0.: 
#             ds = random()*self._random_delay
#             logging.debug(f'CianParser: delay {ds:.2f} sec')
#             sleep( ds )
#         return super()._read_page(url,npage=npage)
    
#     def _is_last_page(self,root,p): 
#         try:
#             pl = root.find('div',{'data-name':'Pagination',}).find_all('li')[-1].text.strip()
#             logging.debug(f'CianParser: page {p}, last page "{pl}"')
#             if pl=='..': return False
#             return p>=int(pl)
#         except Exception as e:
#             logging.warning(f'CianParser: parse pagination error: {e}')
#             return True
        
#     def _parse_item(self,tag): 
#         return { 
#              'OfferTitle': self._get_title(tag),
#           'OfferSubtitle': self._get_subtitle(tag),
#                'Deadline': self._get_deadline(tag),
#               'MainPrice': self._get_price(tag),
#               'PriceInfo': self._get_price_m(tag),
#                'GeoLabel': self._get_adr(tag),
#               'TimeLabel': self._get_ts( tag ),  
#                'LinkArea': self._get_link( tag ),
#             'Description': self._get_descr(tag),
#             }
    
#     @staticmethod
#     def _get_deadline(tag):
#         try:
#             return tag.find('span',{'data-mark':'Deadline',}).text
#         except:
#             return ''
        
#     @staticmethod
#     def _get_title(tag):
#         try:
#             return tag.find('span',{'data-mark':'OfferTitle',}).text
#         except:
#             return ''
        
#     @staticmethod
#     def _get_subtitle(tag):
#         try:
#             return tag.find('span',{'data-mark':'OfferSubtitle',}).text
#         except:
#             return ''
        
#     @staticmethod
#     def _get_price(tag):
#         try:
#             return tag.find('span',{'data-mark':'MainPrice',}).text
#         except:
#             return ''

#     @staticmethod
#     def _get_price_m(tag):
#         try:
#             return tag.find('p',{'data-mark':'PriceInfo',}).text
#         except:
#             return ''
        
#     @staticmethod
#     def _get_adr(tag):
#         try:
#             return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]
#         except:
#             return []
    
#     @staticmethod
#     def _get_ts(tag):
#         try:
#             return tag.find('div',{'data-name':'TimeLabel',}).find_all('span')
#         except:
#             return []
        
#     @staticmethod
#     def _get_link(tag):
#         try:
#             return tag.find('div',{'data-name':'LinkArea',}).find('a').attrs['href']
#         except:
#             return ''
        
#     @staticmethod
#     def _get_descr(tag):
#         try:
#             return tag.find('div',{'data-name':'Description',}).text
#         except:
#             return ''
    